In [21]:
import pandas as pd
import math
import numpy as np
import itertools

In [22]:
#importing targeted receivers and cb_wr dataset from dylan

targetedReceiver = pd.read_csv('/Users/alexmcgraw/Documents/Adv_Predictive_Modeling/APM-Term-Project/targetedReceiver.csv')
cb_wr = pd.read_csv('/Users/alexmcgraw/Documents/Adv_Predictive_Modeling/APM-Term-Project/cb_wr.csv')

mask = ((cb_wr['position'] == 'CB') | (cb_wr['position'] == 'WR') & (cb_wr['event'] == 'pass_forward') | (cb_wr['event']\
                                                                                                        == 'pass_arrived'))
#getting only the pass forward and arrived events for CB and WR
cb_wr_passes = cb_wr[mask]

#only one game
game1 = cb_wr_passes[(cb_wr_passes['gameId'] == 2018090600)]
game1 = game1[['time','playId', 'x', 'y', 'event','nflId','position','team']]

game1Receivers = targetedReceiver[(targetedReceiver['gameId'] == 2018090600)]


#gets rid of scientific notation
pd.set_option('display.float_format', lambda x: '%.0f' % x) 

In [23]:
games = cb_wr_passes['gameId'].unique()

playerEucDiff = pd.DataFrame()

for game in games:
    
    game1 = cb_wr_passes[(cb_wr_passes['gameId'] == game)]
    game1 = game1[['time','playId', 'x', 'y', 'event','nflId','position','team']]

    game1Receivers = targetedReceiver[(targetedReceiver['gameId'] == game)]
    
    game_receiver_merge = game1.merge(game1Receivers, how='outer', left_on = ['playId', 'nflId'], right_on = ['playId',
                                                                                                          'targetNflId'])
    #replacing NaN with None
    game_receiver_merge = game_receiver_merge.where(pd.notnull(game_receiver_merge), None)

    #getting rid of instances where there is a WR that isn't the targeted one
    game_receiver_merge = game_receiver_merge[(((game_receiver_merge['position'] == 'WR') & (game_receiver_merge['nflId'] == game_receiver_merge
                                                                     ['targetNflId'])) | \
                         (game_receiver_merge['position'] == 'CB'))]

    #zipping x and y
    game_receiver_merge['x,y'] = list(zip(game_receiver_merge['x'], game_receiver_merge['y']))

    #dropping x and y columns
    game_receiver_merge = game_receiver_merge.drop(['x', 'y'], axis=1)

    
    #unique play ids
    plays = pd.Series(game_receiver_merge['playId'].unique())

    #creating a new DF, that only has plays that have both a WR and CB, bc sometimes plays have CB but no WR, bc none targeted
    merge_fixed = pd.DataFrame(columns=game_receiver_merge.columns)

    for play in plays:
        df_temp = game_receiver_merge[(game_receiver_merge['playId'] == play)]
        unique_pos = df_temp['position'].unique()
        if len(unique_pos) == 2:
            merge_fixed = merge_fixed.append(df_temp)
            
    
    merged2_fixed = merge_fixed.merge(merge_fixed, on=['time', 'event'])
    
    #keeping rows where the player positions aren't the same (so no pairings between two WR or CB)
    merged2_fixed = merged2_fixed[((merged2_fixed['position_x'] != merged2_fixed['position_y']))]
    
    #Calculating the euclidean distance between the players
    
    def euc_dist(item):
        return math.sqrt(((item['x,y_x'][0] - item['x,y_y'][0])**2) + 
                                         ((item['x,y_x'][1] - item['x,y_y'][1])**2))

    merged2_fixed['Euc_dist'] = merged2_fixed.apply(euc_dist, axis=1)
    
        #cuts df by half. Keeps only rows where it is WR then CB, and not CB and WR (which is just a repeat)
    merged2_fixed = merged2_fixed[(merged2_fixed['position_x'] == 'WR')]
    #print(merged2_fixed.shape)

    mins = []
    #This is dropping farther CBs. Will prob need to fix, bc it doesn't account for the case of 3 CBs and 1 WR. Would 
    # only drop the 1 furthest, not both furthest. 
    for play in plays:
        df_temp = merged2_fixed[((merged2_fixed['playId_x'] == play))]
        #print(df_temp)
        if len(df_temp) > 1:

            idmin = df_temp[(df_temp['event'] == 'pass_forward')]['Euc_dist'].idxmin()
            nflmin = df_temp['nflId_y'].loc[idmin]
            min_ids = df_temp[df_temp['nflId_y'] == nflmin].index.tolist()

            mins.append(min_ids)



    flattened_mins = list(itertools.chain(*mins))

    merged2_fixed = merged2_fixed.loc[flattened_mins]
    
    #self merging AGAIN, so now have a row with the pass_forward and the pass_arrived. Much easier to calculate diff
    ForArrStack = merged2_fixed.merge(merged2_fixed, on = ['playId_x', 'nflId_x', 'nflId_y'], suffixes=['_Xagain', '_Yagain'])
    
    #Get rid of rows where the forward time is equal to or greater than the arrived time
    ForArrStack = ForArrStack[(ForArrStack['time_Xagain'] < ForArrStack['time_Yagain'])]
    
    #cut down on useless rows
    ForArrStack.drop(['time_Yagain', 'targetNflId_y_Yagain', 'position_x_Yagain', 'position_y_Yagain', 'gameId_y_Yagain', 'gameId_x_Yagain', 'playId_y_Yagain', 'gameId_y_Xagain', 'targetNflId_x_Yagain'], axis=1, inplace=True)

    #Calc the Euclidean Difference from pass forward and pass arrived
    ForArrStack['Euc_diff'] = ForArrStack['Euc_dist_Xagain'] - ForArrStack['Euc_dist_Yagain']

    #created a new DF and appended the CB nfl IDs and the Euclidean difference for each play
    playerEucDiff['gameId'] = game
    playerEucDiff['nflId_CB'] = ForArrStack['nflId_y']
    playerEucDiff['Euc_diff'] = ForArrStack['Euc_diff']
    playerEucDiff

ValueError: attempt to get argmin of an empty sequence

In [4]:
game1Receivers.head()

,gameId,playId,targetNflId
0,2018090600,75,2495454.0
1,2018090600,146,2560854.0
2,2018090600,168,2543583.0
3,2018090600,190,2495454.0
4,2018090600,256,2543583.0


In [5]:
#Merging game and targeted receivers
game_receiver_merge = game1.merge(game1Receivers, how='outer', left_on = ['playId', 'nflId'], right_on = ['playId',
                                                                                                          'targetNflId'])
#replacing NaN with None
game_receiver_merge = game_receiver_merge.where(pd.notnull(game_receiver_merge), None)

#getting rid of instances where there is a WR that isn't the targeted one
game_receiver_merge = game_receiver_merge[(((game_receiver_merge['position'] == 'WR') & (game_receiver_merge['nflId'] == game_receiver_merge
                                                                 ['targetNflId'])) | \
                     (game_receiver_merge['position'] == 'CB'))]

#zipping x and y
game_receiver_merge['x,y'] = list(zip(game_receiver_merge['x'], game_receiver_merge['y']))

#dropping x and y columns
game_receiver_merge = game_receiver_merge.drop(['x', 'y'], axis=1)

#gets rid of scientific notation
pd.set_option('display.float_format', lambda x: '%.0f' % x) 

game_receiver_merge.head()


,time,playId,event,nflId,position,team,gameId,targetNflId,"x,y"
0,2018-09-07T01:07:18.099Z,75,pass_forward,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)"
1,2018-09-07T01:07:19.200Z,75,pass_arrived,2495454,WR,away,2018090600,2495454,"(81.11, 47.87)"
4,2018-09-07T01:07:18.099Z,75,pass_forward,2552689,CB,home,None,None,"(79.51, 20.0)"
5,2018-09-07T01:07:19.200Z,75,pass_arrived,2552689,CB,home,None,None,"(77.11, 17.3)"
6,2018-09-07T01:07:18.099Z,75,pass_forward,2555383,CB,home,None,None,"(76.53, 44.93)"


In [6]:
# FIX SO WHEN ITERATING THRU EVERY GAME

#unique play ids
plays = pd.Series(game_receiver_merge['playId'].unique())

#creating a new DF, that only has plays that have both a WR and CB, bc sometimes plays have CB but no WR, bc none targeted
merge_fixed = pd.DataFrame(columns=game_receiver_merge.columns)

for play in plays:
    df_temp = game_receiver_merge[(game_receiver_merge['playId'] == play)]
    unique_pos = df_temp['position'].unique()
    if len(unique_pos) == 2:
        merge_fixed = merge_fixed.append(df_temp)
        
merge_fixed        


,time,playId,event,nflId,position,team,gameId,targetNflId,"x,y"
0,2018-09-07T01:07:18.099Z,75,pass_forward,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)"
1,2018-09-07T01:07:19.200Z,75,pass_arrived,2495454,WR,away,2018090600,2495454,"(81.11, 47.87)"
4,2018-09-07T01:07:18.099Z,75,pass_forward,2552689,CB,home,None,None,"(79.51, 20.0)"
5,2018-09-07T01:07:19.200Z,75,pass_arrived,2552689,CB,home,None,None,"(77.11, 17.3)"
6,2018-09-07T01:07:18.099Z,75,pass_forward,2555383,CB,home,None,None,"(76.53, 44.93)"
7,2018-09-07T01:07:19.200Z,75,pass_arrived,2555383,CB,home,None,None,"(74.4, 47.11)"
10,2018-09-07T01:09:12.700Z,146,pass_forward,2552689,CB,home,None,None,"(38.1, 10.36)"
11,2018-09-07T01:09:13.200Z,146,pass_arrived,2552689,CB,home,None,None,"(37.4, 9.64)"
12,2018-09-07T01:09:12.700Z,146,pass_forward,2555383,CB,home,None,None,"(41.28, 38.93)"
13,2018-09-07T01:09:13.200Z,146,pass_arrived,2555383,CB,home,None,None,"(43.3, 37.96)"


In [7]:
#merging on itself
merged2_fixed = merge_fixed.merge(merge_fixed, on=['time', 'event'])
merged2_fixed.head()


,time,playId_x,event,nflId_x,position_x,team_x,gameId_x,targetNflId_x,"x,y_x",playId_y,nflId_y,position_y,team_y,gameId_y,targetNflId_y,"x,y_y"
0,2018-09-07T01:07:18.099Z,75,pass_forward,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)",75,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)"
1,2018-09-07T01:07:18.099Z,75,pass_forward,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)",75,2552689,CB,home,None,None,"(79.51, 20.0)"
2,2018-09-07T01:07:18.099Z,75,pass_forward,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)",75,2555383,CB,home,None,None,"(76.53, 44.93)"
3,2018-09-07T01:07:18.099Z,75,pass_forward,2552689,CB,home,None,None,"(79.51, 20.0)",75,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)"
4,2018-09-07T01:07:18.099Z,75,pass_forward,2552689,CB,home,None,None,"(79.51, 20.0)",75,2552689,CB,home,None,None,"(79.51, 20.0)"


In [8]:
#keeping rows where the player positions aren't the same (so no pairings between two WR or CB)
merged2_fixed = merged2_fixed[((merged2_fixed['position_x'] != merged2_fixed['position_y']))]
      
    
merged2_fixed.iloc[:10]


,time,playId_x,event,nflId_x,position_x,team_x,gameId_x,targetNflId_x,"x,y_x",playId_y,nflId_y,position_y,team_y,gameId_y,targetNflId_y,"x,y_y"
1,2018-09-07T01:07:18.099Z,75,pass_forward,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)",75,2552689,CB,home,None,None,"(79.51, 20.0)"
2,2018-09-07T01:07:18.099Z,75,pass_forward,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)",75,2555383,CB,home,None,None,"(76.53, 44.93)"
3,2018-09-07T01:07:18.099Z,75,pass_forward,2552689,CB,home,None,None,"(79.51, 20.0)",75,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)"
6,2018-09-07T01:07:18.099Z,75,pass_forward,2555383,CB,home,None,None,"(76.53, 44.93)",75,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)"
10,2018-09-07T01:07:19.200Z,75,pass_arrived,2495454,WR,away,2018090600,2495454,"(81.11, 47.87)",75,2552689,CB,home,None,None,"(77.11, 17.3)"
11,2018-09-07T01:07:19.200Z,75,pass_arrived,2495454,WR,away,2018090600,2495454,"(81.11, 47.87)",75,2555383,CB,home,None,None,"(74.4, 47.11)"
12,2018-09-07T01:07:19.200Z,75,pass_arrived,2552689,CB,home,None,None,"(77.11, 17.3)",75,2495454,WR,away,2018090600,2495454,"(81.11, 47.87)"
15,2018-09-07T01:07:19.200Z,75,pass_arrived,2555383,CB,home,None,None,"(74.4, 47.11)",75,2495454,WR,away,2018090600,2495454,"(81.11, 47.87)"
21,2018-09-07T01:09:12.700Z,146,pass_forward,2552689,CB,home,None,None,"(38.1, 10.36)",146,2560854,WR,away,2018090600,2560854,"(45.51, 36.91)"
25,2018-09-07T01:09:12.700Z,146,pass_forward,2555383,CB,home,None,None,"(41.28, 38.93)",146,2560854,WR,away,2018090600,2560854,"(45.51, 36.91)"


In [9]:
#Calculating the euclidean distance between the players
    
def euc_dist(item):
    return math.sqrt(((item['x,y_x'][0] - item['x,y_y'][0])**2) + 
                                     ((item['x,y_x'][1] - item['x,y_y'][1])**2))

merged2_fixed['Euc_dist'] = merged2_fixed.apply(euc_dist, axis=1)
    
merged2_fixed.head(25)


,time,playId_x,event,nflId_x,position_x,team_x,gameId_x,targetNflId_x,"x,y_x",playId_y,nflId_y,position_y,team_y,gameId_y,targetNflId_y,"x,y_y",Euc_dist
1,2018-09-07T01:07:18.099Z,75,pass_forward,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)",75,2552689,CB,home,None,None,"(79.51, 20.0)",25
2,2018-09-07T01:07:18.099Z,75,pass_forward,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)",75,2555383,CB,home,None,None,"(76.53, 44.93)",4
3,2018-09-07T01:07:18.099Z,75,pass_forward,2552689,CB,home,None,None,"(79.51, 20.0)",75,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)",25
6,2018-09-07T01:07:18.099Z,75,pass_forward,2555383,CB,home,None,None,"(76.53, 44.93)",75,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)",4
10,2018-09-07T01:07:19.200Z,75,pass_arrived,2495454,WR,away,2018090600,2495454,"(81.11, 47.87)",75,2552689,CB,home,None,None,"(77.11, 17.3)",31
11,2018-09-07T01:07:19.200Z,75,pass_arrived,2495454,WR,away,2018090600,2495454,"(81.11, 47.87)",75,2555383,CB,home,None,None,"(74.4, 47.11)",7
12,2018-09-07T01:07:19.200Z,75,pass_arrived,2552689,CB,home,None,None,"(77.11, 17.3)",75,2495454,WR,away,2018090600,2495454,"(81.11, 47.87)",31
15,2018-09-07T01:07:19.200Z,75,pass_arrived,2555383,CB,home,None,None,"(74.4, 47.11)",75,2495454,WR,away,2018090600,2495454,"(81.11, 47.87)",7
21,2018-09-07T01:09:12.700Z,146,pass_forward,2552689,CB,home,None,None,"(38.1, 10.36)",146,2560854,WR,away,2018090600,2560854,"(45.51, 36.91)",28
25,2018-09-07T01:09:12.700Z,146,pass_forward,2555383,CB,home,None,None,"(41.28, 38.93)",146,2560854,WR,away,2018090600,2560854,"(45.51, 36.91)",5


In [10]:
merged2_fixed.head(10)

,time,playId_x,event,nflId_x,position_x,team_x,gameId_x,targetNflId_x,"x,y_x",playId_y,nflId_y,position_y,team_y,gameId_y,targetNflId_y,"x,y_y",Euc_dist
1,2018-09-07T01:07:18.099Z,75,pass_forward,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)",75,2552689,CB,home,None,None,"(79.51, 20.0)",25
2,2018-09-07T01:07:18.099Z,75,pass_forward,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)",75,2555383,CB,home,None,None,"(76.53, 44.93)",4
3,2018-09-07T01:07:18.099Z,75,pass_forward,2552689,CB,home,None,None,"(79.51, 20.0)",75,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)",25
6,2018-09-07T01:07:18.099Z,75,pass_forward,2555383,CB,home,None,None,"(76.53, 44.93)",75,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)",4
10,2018-09-07T01:07:19.200Z,75,pass_arrived,2495454,WR,away,2018090600,2495454,"(81.11, 47.87)",75,2552689,CB,home,None,None,"(77.11, 17.3)",31
11,2018-09-07T01:07:19.200Z,75,pass_arrived,2495454,WR,away,2018090600,2495454,"(81.11, 47.87)",75,2555383,CB,home,None,None,"(74.4, 47.11)",7
12,2018-09-07T01:07:19.200Z,75,pass_arrived,2552689,CB,home,None,None,"(77.11, 17.3)",75,2495454,WR,away,2018090600,2495454,"(81.11, 47.87)",31
15,2018-09-07T01:07:19.200Z,75,pass_arrived,2555383,CB,home,None,None,"(74.4, 47.11)",75,2495454,WR,away,2018090600,2495454,"(81.11, 47.87)",7
21,2018-09-07T01:09:12.700Z,146,pass_forward,2552689,CB,home,None,None,"(38.1, 10.36)",146,2560854,WR,away,2018090600,2560854,"(45.51, 36.91)",28
25,2018-09-07T01:09:12.700Z,146,pass_forward,2555383,CB,home,None,None,"(41.28, 38.93)",146,2560854,WR,away,2018090600,2560854,"(45.51, 36.91)",5


In [11]:
#cuts df by half. Keeps only rows where it is WR then CB, and not CB and WR (which is just a repeat)
merged2_fixed = merged2_fixed[(merged2_fixed['position_x'] == 'WR')]
#print(merged2_fixed.shape)

mins = []
#This is dropping farther CBs. Will prob need to fix, bc it doesn't account for the case of 3 CBs and 1 WR. Would 
# only drop the 1 furthest, not both furthest. 
for play in plays:
    df_temp = merged2_fixed[((merged2_fixed['playId_x'] == play))]
    #print(df_temp)
    if len(df_temp) > 1:
        
        idmin = df_temp[(df_temp['event'] == 'pass_forward')]['Euc_dist'].idxmin()
        nflmin = df_temp['nflId_y'].loc[idmin]
        min_ids = df_temp[df_temp['nflId_y'] == nflmin].index.tolist()

        mins.append(min_ids)



flattened_mins = list(itertools.chain(*mins))

merged2_fixed = merged2_fixed.loc[flattened_mins]


In [12]:
merged2_fixed.head(15)

,time,playId_x,event,nflId_x,position_x,team_x,gameId_x,targetNflId_x,"x,y_x",playId_y,nflId_y,position_y,team_y,gameId_y,targetNflId_y,"x,y_y",Euc_dist
2,2018-09-07T01:07:18.099Z,75,pass_forward,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)",75,2555383,CB,home,None,None,"(76.53, 44.93)",4
11,2018-09-07T01:07:19.200Z,75,pass_arrived,2495454,WR,away,2018090600,2495454,"(81.11, 47.87)",75,2555383,CB,home,None,None,"(74.4, 47.11)",7
31,2018-09-07T01:09:12.700Z,146,pass_forward,2560854,WR,away,2018090600,2560854,"(45.51, 36.91)",146,2555383,CB,home,None,None,"(41.28, 38.93)",5
47,2018-09-07T01:09:13.200Z,146,pass_arrived,2560854,WR,away,2018090600,2560854,"(44.79, 36.79)",146,2555383,CB,home,None,None,"(43.3, 37.96)",2
53,2018-09-07T01:10:38.400Z,190,pass_forward,2495454,WR,away,2018090600,2495454,"(35.7, 21.07)",190,2557958,CB,home,None,None,"(40.19, 17.44)",6
69,2018-09-07T01:10:40.099Z,190,pass_arrived,2495454,WR,away,2018090600,2495454,"(25.99, 12.13)",190,2557958,CB,home,None,None,"(33.23, 14.1)",8
91,2018-09-07T01:16:54.900Z,320,pass_forward,2552600,WR,home,2018090600,2552600,"(16.54, 41.86)",320,2539653,CB,away,None,None,"(20.45, 42.65)",4
107,2018-09-07T01:16:55.700Z,320,pass_arrived,2552600,WR,home,2018090600,2552600,"(17.58, 46.78)",320,2539653,CB,away,None,None,"(18.99, 46.63)",1
115,2018-09-07T01:21:50.000Z,402,pass_forward,2533040,WR,away,2018090600,2533040,"(36.12, 13.32)",402,2552689,CB,home,None,None,"(32.56, 13.27)",4
131,2018-09-07T01:21:51.299Z,402,pass_arrived,2533040,WR,away,2018090600,2533040,"(37.11, 13.36)",402,2552689,CB,home,None,None,"(35.3, 12.29)",2


In [13]:
#self merging AGAIN, so now have a row with the pass_forward and the pass_arrived. Much easier to calculate diff
ForArrStack = merged2_fixed.merge(merged2_fixed, on = ['playId_x', 'nflId_x', 'nflId_y'], suffixes=['_Xagain', '_Yagain'])
ForArrStack.columns


Index(['time_Xagain', 'playId_x', 'event_Xagain', 'nflId_x',
       'position_x_Xagain', 'team_x_Xagain', 'gameId_x_Xagain',
       'targetNflId_x_Xagain', 'x,y_x_Xagain', 'playId_y_Xagain', 'nflId_y',
       'position_y_Xagain', 'team_y_Xagain', 'gameId_y_Xagain',
       'targetNflId_y_Xagain', 'x,y_y_Xagain', 'Euc_dist_Xagain',
       'time_Yagain', 'event_Yagain', 'position_x_Yagain', 'team_x_Yagain',
       'gameId_x_Yagain', 'targetNflId_x_Yagain', 'x,y_x_Yagain',
       'playId_y_Yagain', 'position_y_Yagain', 'team_y_Yagain',
       'gameId_y_Yagain', 'targetNflId_y_Yagain', 'x,y_y_Yagain',
       'Euc_dist_Yagain'],
      dtype='object')

In [14]:
#Get rid of rows where the forward time is equal to or greater than the arrived time
ForArrStack = ForArrStack[(ForArrStack['time_Xagain'] < ForArrStack['time_Yagain'])]

In [15]:
#cut down on useless rows
print(ForArrStack.iloc[10])
ForArrStack.drop(['time_Yagain', 'targetNflId_y_Yagain', 'position_x_Yagain', 'position_y_Yagain', 'gameId_y_Yagain', 'gameId_x_Yagain', 'playId_y_Yagain', 'gameId_y_Xagain', 'targetNflId_x_Yagain'], axis=1, inplace=True)
print(ForArrStack.shape)

time_Xagain             2018-09-07T01:53:24.000Z
playId_x                                    1226
event_Xagain                        pass_forward
nflId_x                                  2507763
position_x_Xagain                             WR
team_x_Xagain                               home
gameId_x_Xagain                       2018090600
targetNflId_x_Xagain                     2507763
x,y_x_Xagain                      (20.75, 42.48)
playId_y_Xagain                             1226
nflId_y                                  2539334
position_y_Xagain                             CB
team_y_Xagain                               away
gameId_y_Xagain                             None
targetNflId_y_Xagain                        None
x,y_y_Xagain                       (19.8, 43.57)
Euc_dist_Xagain                                1
time_Yagain             2018-09-07T01:53:25.000Z
event_Yagain                        pass_arrived
position_x_Yagain                             WR
team_x_Yagain       

In [16]:
#Calc the Euclidean Difference from pass forward and pass arrived
ForArrStack['Euc_diff'] = ForArrStack['Euc_dist_Xagain'] - ForArrStack['Euc_dist_Yagain']
ForArrStack.head()

,time_Xagain,playId_x,event_Xagain,nflId_x,position_x_Xagain,team_x_Xagain,gameId_x_Xagain,targetNflId_x_Xagain,"x,y_x_Xagain",playId_y_Xagain,...,targetNflId_y_Xagain,"x,y_y_Xagain",Euc_dist_Xagain,event_Yagain,team_x_Yagain,"x,y_x_Yagain",team_y_Yagain,"x,y_y_Yagain",Euc_dist_Yagain,Euc_diff
1,2018-09-07T01:07:18.099Z,75,pass_forward,2495454,WR,away,2018090600,2495454,"(80.69, 44.91)",75,...,None,"(76.53, 44.93)",4,pass_arrived,away,"(81.11, 47.87)",home,"(74.4, 47.11)",7,-3
5,2018-09-07T01:09:12.700Z,146,pass_forward,2560854,WR,away,2018090600,2560854,"(45.51, 36.91)",146,...,None,"(41.28, 38.93)",5,pass_arrived,away,"(44.79, 36.79)",home,"(43.3, 37.96)",2,3
9,2018-09-07T01:10:38.400Z,190,pass_forward,2495454,WR,away,2018090600,2495454,"(35.7, 21.07)",190,...,None,"(40.19, 17.44)",6,pass_arrived,away,"(25.99, 12.13)",home,"(33.23, 14.1)",8,-2
13,2018-09-07T01:16:54.900Z,320,pass_forward,2552600,WR,home,2018090600,2552600,"(16.54, 41.86)",320,...,None,"(20.45, 42.65)",4,pass_arrived,home,"(17.58, 46.78)",away,"(18.99, 46.63)",1,3
17,2018-09-07T01:21:50.000Z,402,pass_forward,2533040,WR,away,2018090600,2533040,"(36.12, 13.32)",402,...,None,"(32.56, 13.27)",4,pass_arrived,away,"(37.11, 13.36)",home,"(35.3, 12.29)",2,1


In [17]:
#created a new DF and appended the CB nfl IDs and the Euclidean difference for each play
playerEucDiff = pd.DataFrame()
playerEucDiff['nflId_CB'] = ForArrStack['nflId_y']
playerEucDiff['Euc_diff'] = ForArrStack['Euc_diff']
playerEucDiff

,nflId_CB,Euc_diff
1,2555383,-3
5,2555383,3
9,2557958,-2
13,2539653,3
17,2552689,1
21,2539653,2
26,2539334,-2
30,2539653,1
34,2539653,0
38,2556445,-0


In [18]:
# Gotta iterate through each game. Also, note on the CB, WR pairing code. Last, need to do these as percentages, so
# is not skewed by scale of the coverage distances